In [75]:
from scipy.io import arff
import pandas as pd
import numpy as np 

In [76]:
data = arff.loadarff('veh-prime.arff')
df = pd.DataFrame(data[0])
df['CLASS'][df['CLASS']==b'car']=1
df['CLASS'][df['CLASS']==b'noncar']=0

C:\Users\scum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\scum\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [77]:
def pearson(x,y):
    N=845
    sum_sq_x = 0
    sum_sq_y = 0
    sum_coproduct = 0
    mean_x = 0
    mean_y = 0
    for i in range(N) :
        sum_sq_x += x[i] * x[i]
        sum_sq_y += y[i] * y[i]
        sum_coproduct += x[i] * y[i]
        mean_x += x[i]
        mean_y += y[i]
    mean_x = mean_x / N
    mean_y = mean_y / N
    pop_sd_x = ((sum_sq_x/N) - (mean_x * mean_x))**0.5
    pop_sd_y = ((sum_sq_y / N) - (mean_y * mean_y))**0.5
    cov_x_y = (sum_coproduct / N) - (mean_x * mean_y)
    correlation = cov_x_y / (pop_sd_x * pop_sd_y)
    return correlation
        
def distance(train,test):
    m=train.shape[1]-1
    d=np.sqrt(np.dot(np.square(np.subtract(np.dot(np.ones((train.shape[0],1)),[test[:-1]]),train[:,:-1])),np.ones((m,1))).tolist()).tolist()
    for i in range(train.shape[0]):
        d[i].append(train[i][-1])
    d.sort()
    return d

def percent(data):
    right=0
    wrong=0
    n=data.shape[0]
    for j in range(n):
        a=list(range(n))
        a.remove(j)
        train=data[a]
        test=data[j]
        d=distance(train,test)
        car=0
        nocar=0
        for k in range(7):
            if d[k][1]==1:
                car=car+1
            else:
                nocar=nocar+1
        pred=1 if (car>nocar) else 0
        if pred==data[j][-1]:
            right=right+1
        else:
            wrong=wrong+1
    return right/(right+wrong)

def filt(df,r):
    df=df.to_numpy()
    results=[]
    features=[-1]
    for i in range(36):
        features.remove(-1)
        features.append(r[i][1])
        features.append(-1)
        results.append([i+1,percent(df[:,features])])

        print(results[i])
    return results

def wrapper(df):
    accuracy=0
    df=df.to_numpy()
    index=list(range(36))
    ma=-1
    results=[]
    features=[]
    for i in range(36):
        found=False
        for j in index:
            features.append(j)
            features.append(-1)
            try_data=df[:,features]
            new_accuracy=percent(try_data)
            print(j,new_accuracy)
            if new_accuracy>accuracy:
                accuracy=new_accuracy
                ma=j
                found=True
            features.remove(-1)
            features.remove(j)
        if found:
            features.append(ma)
            index.remove(ma)
            results.append([ma,accuracy])
            print(results)
        else:
            break
    return results
        

In [68]:
r=[]
for i in range(36):
    r.append([abs(pearson(df['f'+str(i)],df['CLASS'])),i])
r.sort(reverse=True)
r

[[0.43623110241592233, 4],
 [0.3671977612614123, 13],
 [0.36693984833130056, 14],
 [0.3650005209473775, 16],
 [0.3510981014201862, 7],
 [0.35035926985651833, 22],
 [0.34116492222041855, 26],
 [0.307934982102883, 1],
 [0.2977932634946587, 20],
 [0.29175038412300547, 31],
 [0.2654353063184128, 34],
 [0.19436342728957748, 2],
 [0.15579674817511585, 28],
 [0.15173750447344872, 25],
 [0.13621735048491446, 19],
 [0.11372281831570262, 17],
 [0.09267086026183788, 32],
 [0.08888468881807043, 8],
 [0.06862742861915738, 0],
 [0.05599972384141407, 10],
 [0.05582293533321709, 21],
 [0.04218561762173958, 11],
 [0.0384277321447869, 33],
 [0.037777122895097, 6],
 [0.03262548472949429, 15],
 [0.02984210815092925, 35],
 [0.017956953579203145, 18],
 [0.017866084905459306, 29],
 [0.01563553687994917, 27],
 [0.01298534239184928, 9],
 [0.010326604649079408, 30],
 [0.008561274009635453, 24],
 [0.008421432416288488, 3],
 [0.003679663149250195, 23],
 [0.0021741048783563773, 12],
 [0.001173877055453107, 5]]

In [78]:
filt(df,r)

[1, 0.6702127659574468]
[2, 0.7978723404255319]
[3, 0.8238770685579196]
[4, 0.8333333333333334]
[5, 0.83451536643026]
[6, 0.8368794326241135]
[7, 0.8617021276595744]
[8, 0.8747044917257684]
[9, 0.8947990543735225]
[10, 0.8853427895981087]
[11, 0.9018912529550828]
[12, 0.9042553191489362]
[13, 0.8888888888888888]
[14, 0.8912529550827423]
[15, 0.9089834515366431]
[16, 0.9160756501182034]
[17, 0.933806146572104]
[18, 0.9468085106382979]
[19, 0.942080378250591]
[20, 0.950354609929078]
[21, 0.9408983451536643]
[22, 0.9432624113475178]
[23, 0.9456264775413712]
[24, 0.9444444444444444]
[25, 0.9361702127659575]
[26, 0.9408983451536643]
[27, 0.9468085106382979]
[28, 0.942080378250591]
[29, 0.9456264775413712]
[30, 0.9432624113475178]
[31, 0.9361702127659575]
[32, 0.9397163120567376]
[33, 0.942080378250591]
[34, 0.9432624113475178]
[35, 0.9314420803782506]
[36, 0.9373522458628841]


[[1, 0.6702127659574468],
 [2, 0.7978723404255319],
 [3, 0.8238770685579196],
 [4, 0.8333333333333334],
 [5, 0.83451536643026],
 [6, 0.8368794326241135],
 [7, 0.8617021276595744],
 [8, 0.8747044917257684],
 [9, 0.8947990543735225],
 [10, 0.8853427895981087],
 [11, 0.9018912529550828],
 [12, 0.9042553191489362],
 [13, 0.8888888888888888],
 [14, 0.8912529550827423],
 [15, 0.9089834515366431],
 [16, 0.9160756501182034],
 [17, 0.933806146572104],
 [18, 0.9468085106382979],
 [19, 0.942080378250591],
 [20, 0.950354609929078],
 [21, 0.9408983451536643],
 [22, 0.9432624113475178],
 [23, 0.9456264775413712],
 [24, 0.9444444444444444],
 [25, 0.9361702127659575],
 [26, 0.9408983451536643],
 [27, 0.9468085106382979],
 [28, 0.942080378250591],
 [29, 0.9456264775413712],
 [30, 0.9432624113475178],
 [31, 0.9361702127659575],
 [32, 0.9397163120567376],
 [33, 0.942080378250591],
 [34, 0.9432624113475178],
 [35, 0.9314420803782506],
 [36, 0.9373522458628841]]

In [72]:
wrapper(df) # with normalization

0 0.5070921985815603
1 0.5851063829787234
2 0.5118203309692672
3 0.49645390070921985
4 0.6702127659574468
5 0.49645390070921985
6 0.475177304964539
7 0.6442080378250591
8 0.4929078014184397
9 0.508274231678487
10 0.524822695035461
11 0.46099290780141844
12 0.5141843971631206
13 0.7328605200945626
14 0.475177304964539
15 0.516548463356974
16 0.48817966903073284
17 0.49645390070921985
18 0.5106382978723404
19 0.6063829787234043
20 0.7328605200945626
21 0.5118203309692672
22 0.7375886524822695
23 0.5094562647754137
24 0.5
25 0.6442080378250591
26 0.5047281323877069
27 0.5189125295508275
28 0.5094562647754137
29 0.5141843971631206
30 0.533096926713948
31 0.5862884160756501
32 0.5
33 0.5177304964539007
34 0.5
35 0.5106382978723404
[[22, 0.7375886524822695]]
0 0.7316784869976359
1 0.764775413711584
2 0.7919621749408984
3 0.7222222222222222
4 0.8096926713947991
5 0.7316784869976359
6 0.7293144208037825
7 0.776595744680851
8 0.8132387706855791
9 0.7186761229314421
10 0.859338061465721
11 0.754

[[22, 0.7375886524822695],
 [10, 0.859338061465721],
 [19, 0.8912529550827423],
 [8, 0.9243498817966903],
 [7, 0.9491725768321513],
 [13, 0.9515366430260047],
 [1, 0.9550827423167849],
 [25, 0.958628841607565]]

In [74]:
wrapper(df) # without normalization

0 0.5094562647754137
1 0.5851063829787234
2 0.5118203309692672
3 0.4940898345153664
4 0.6702127659574468
5 0.4988179669030733
6 0.47754137115839246
7 0.6430260047281324
8 0.4929078014184397
9 0.5047281323877069
10 0.524822695035461
11 0.458628841607565
12 0.5141843971631206
13 0.735224586288416
14 0.475177304964539
15 0.524822695035461
16 0.48817966903073284
17 0.49645390070921985
18 0.5118203309692672
19 0.6063829787234043
20 0.7304964539007093
21 0.508274231678487
22 0.7387706855791962
23 0.5094562647754137
24 0.5
25 0.6477541371158393
26 0.5047281323877069
27 0.5212765957446809
28 0.5094562647754137
29 0.5141843971631206
30 0.5401891252955082
31 0.5862884160756501
32 0.49527186761229314
33 0.5141843971631206
34 0.5
35 0.5070921985815603
[[22, 0.7387706855791962]]
0 0.7411347517730497
1 0.764775413711584
2 0.7919621749408984
3 0.7328605200945626
4 0.8096926713947991
5 0.7316784869976359
6 0.7281323877068558
7 0.776595744680851
8 0.8132387706855791
9 0.735224586288416
10 0.85933806146

[[22, 0.7387706855791962],
 [10, 0.859338061465721],
 [19, 0.8912529550827423],
 [8, 0.9243498817966903],
 [7, 0.9491725768321513],
 [13, 0.9515366430260047],
 [1, 0.9550827423167849],
 [25, 0.958628841607565],
 [9, 0.9598108747044918]]

In [198]:
a[1]

[0.002653633739662681, 0]

In [172]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f27,f28,f29,f30,f31,f32,f33,f34,f35,CLASS
0,0.063,0.160,0.509,-0.967,0.058,0.000,0.874,0.271,1.307,-0.011,...,-0.924,-0.077,0.108,-0.003,0.381,-0.314,0.929,0.184,-0.001,0
1,-0.037,-0.325,-0.626,-0.029,0.121,-0.409,-0.002,-0.835,-0.595,-0.253,...,0.270,0.533,0.152,-0.978,0.157,0.011,-0.254,0.453,-0.621,0
2,-0.000,1.253,0.833,-0.970,1.516,0.014,-0.378,1.197,0.546,-0.402,...,-0.408,1.550,0.010,-0.652,-0.403,-0.151,0.000,0.049,-0.113,1
3,-0.743,-0.082,-0.626,0.723,-0.006,-0.000,-0.080,-0.297,0.166,0.311,...,0.819,-0.077,-0.099,-0.001,-0.291,1.633,0.686,1.528,-0.000,0
4,-0.939,-1.054,-0.140,0.036,-0.766,0.000,-0.272,1.077,5.236,-0.366,...,0.676,0.533,-0.003,0.122,-0.179,-1.449,0.024,-1.698,0.083,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,0.072,-0.082,-0.950,0.359,0.311,0.368,-0.259,0.420,0.292,-0.070,...,0.005,0.127,0.380,-0.001,1.388,-0.151,0.291,-0.085,0.000,1
842,-0.555,-0.568,0.184,-0.920,0.121,0.883,-0.006,-0.177,0.546,0.282,...,0.039,-1.093,0.066,0.002,0.829,-0.476,0.924,0.184,0.170,0
843,-0.703,1.496,1.481,-0.529,1.199,-0.085,-0.980,1.585,0.673,-0.553,...,-0.704,-0.687,0.059,-0.294,-0.963,-0.314,-0.204,0.722,-0.386,1
844,0.296,-0.932,-1.436,0.113,-0.259,0.905,0.409,-0.685,-0.468,-0.053,...,-0.156,-1.297,-0.238,-0.000,1.388,0.173,0.016,-0.085,0.873,1


In [174]:
b=df[["f0","CLASS"]].to_numpy()
a=percent(b)


In [175]:
a

0.5035460992907801

In [179]:
a=(b[:,0]-b[:,0].mean())/b[:,0].std()

In [180]:
a

array([0.12305801722175777, -0.14247385864124398, -0.044227064571933344,
       -2.0171289022340364, -2.5375713789255196, -0.7000907979535477,
       -0.044227064571933344, 1.7162492723997684, -0.04157174581330332,
       -0.17168236498617417, 0.5346324248094105, 0.7364366504652918,
       1.6950067223307284, 1.5728620594337475, -0.3389674467798653,
       0.032777179428337175, -1.2895715623694115, -1.7064566074743244,
       1.1028706391562344, -0.5274950786425966, 2.5447087250923337,
       -0.028295152020153234, 0.006223991842036993, -0.03095047077878325,
       1.1692536081219849, 0.7948536631551523, -0.04157174581330332,
       -1.7701842576814448, -0.044227064571933344, 0.33813883667078914,
       -0.060158977123713445, -1.8179799953367852, -1.03997159905819,
       -1.8232906328540452, -0.044227064571933344, -0.044227064571933344,
       0.35938138673982933, 0.30627501156722897, 0.46559413708503006,
       -0.04688238333056336, 0.8692025883967928, -0.0229845145028932,
       -0.

In [46]:
m=[]
std=[]
for i in range(36):
    m.append(df["f"+str(i)].mean())
    std.append(df["f"+str(i)].std())

In [48]:
a=pd.DataFrame([])
a["std"]=std
a["mean"]=m
a

,std,mean
0,0.376825,1.665603e-02
1,0.999968,1.536643e-05
2,0.999930,-4.373522e-05
3,0.381564,1.990544e-03
4,0.999995,6.737589e-05
5,0.359452,-1.956856e-02
6,0.392873,-3.903073e-03
7,1.000021,1.891253e-05
8,1.000011,-4.964539e-05
9,0.375906,5.255319e-03


In [60]:
n=[1,2,3]
b=n

In [3]:
def t(n):
    n[0]=5

In [61]:
del n[:]

In [62]:
b

[]

In [64]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f27,f28,f29,f30,f31,f32,f33,f34,f35,CLASS
0,0.063,0.160,0.509,-0.967,0.058,0.000,0.874,0.271,1.307,-0.011,...,-0.924,-0.077,0.108,-0.003,0.381,-0.314,0.929,0.184,-0.001,0
1,-0.037,-0.325,-0.626,-0.029,0.121,-0.409,-0.002,-0.835,-0.595,-0.253,...,0.270,0.533,0.152,-0.978,0.157,0.011,-0.254,0.453,-0.621,0
2,-0.000,1.253,0.833,-0.970,1.516,0.014,-0.378,1.197,0.546,-0.402,...,-0.408,1.550,0.010,-0.652,-0.403,-0.151,0.000,0.049,-0.113,1
3,-0.743,-0.082,-0.626,0.723,-0.006,-0.000,-0.080,-0.297,0.166,0.311,...,0.819,-0.077,-0.099,-0.001,-0.291,1.633,0.686,1.528,-0.000,0
4,-0.939,-1.054,-0.140,0.036,-0.766,0.000,-0.272,1.077,5.236,-0.366,...,0.676,0.533,-0.003,0.122,-0.179,-1.449,0.024,-1.698,0.083,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,0.072,-0.082,-0.950,0.359,0.311,0.368,-0.259,0.420,0.292,-0.070,...,0.005,0.127,0.380,-0.001,1.388,-0.151,0.291,-0.085,0.000,1
842,-0.555,-0.568,0.184,-0.920,0.121,0.883,-0.006,-0.177,0.546,0.282,...,0.039,-1.093,0.066,0.002,0.829,-0.476,0.924,0.184,0.170,0
843,-0.703,1.496,1.481,-0.529,1.199,-0.085,-0.980,1.585,0.673,-0.553,...,-0.704,-0.687,0.059,-0.294,-0.963,-0.314,-0.204,0.722,-0.386,1
844,0.296,-0.932,-1.436,0.113,-0.259,0.905,0.409,-0.685,-0.468,-0.053,...,-0.156,-1.297,-0.238,-0.000,1.388,0.173,0.016,-0.085,0.873,1


In [36]:
b

[6]

In [37]:
n

[1, 2, 3]

In [116]:
a

array([[5],
       [2]], dtype=int64)

In [117]:
b

array([[5],
       [2]], dtype=int64)

In [100]:
df["f13"]

0     -0.206
1     -0.597
2      1.148
3     -0.747
4     -0.597
       ...  
841    0.005
842   -0.296
843    1.599
844   -1.018
845   -1.469
Name: f13, Length: 846, dtype: float64